In [42]:
# import weave
# weave.init("graphrag-poc")

weave version 0.51.23 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: vedmanivaidya.
View Weave data at https://wandb.ai/vedmanivaidya/graphrag-poc/weave


In [24]:
# Enable auto-reloading of modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import logging
 
# Configure logging to write to a file, setting the level to INFO
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    filename='app.log',  # Log messages are written to this file
                    filemode='a')  # Append mode
 
# Create a StreamHandler to output logs to the console as well
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)  # Set the log level for the console
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
 
# Add the console handler to the root logger
logging.getLogger('').addHandler(console_handler)

In [3]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding


In [4]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
openai_llm = AzureOpenAI(
    model=os.getenv("DEPLOYMENT_NAME"),
    deployment_name=os.getenv("DEPLOYMENT_NAME"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    temperature=0.0,
)
openai_embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    # deployment_name="my-custom-embedding",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("OPENAI_API_VERSION"),
)
embeddings = openai_embed_model.get_text_embedding("Hello, world!")
len(embeddings)

1536

In [5]:
gemini_llm = Gemini(
    model="models/gemini-1.5-flash-latest",
    temperature=0.0,
)
gemini_embed_model = GeminiEmbedding()
embeddings = gemini_embed_model.get_text_embedding("Hello, world!")
len(embeddings)

768

In [6]:
from llama_index.core import Settings

Settings.llm = gemini_llm
Settings.embed_model = gemini_embed_model

In [7]:
import nest_asyncio

nest_asyncio.apply()

In [8]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    "D:\projects\graphrag-poc\data\selection 1"
).load_data()

In [9]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="12345678",
    url="bolt://localhost:7687",
    database="graphrag-openai-custom-extractor-prompt-gemini",
    refresh_schema=False,
)

Verifying version


In [10]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=50,
)
nodes = splitter.get_nodes_from_documents(documents)
len(nodes)
# create a list of lists each of 10 nodes
nodes_list = [nodes[i : i + 10] for i in range(0, len(nodes), 10)]
print(len(nodes_list))

8


In [11]:
prompt_template = (
    "You are a knowledge graph expert specializing in extracting structured information from text about startup founders, companies, and entrepreneurship guidance, particularly from Paul Graham's writings."
    "\nYour task is to extract up to {max_knowledge_triplets} knowledge triplets from the provided text. "
    "\n If there are more triplets that can be extracted, then extract as many as needed to capture all the information"
    "A knowledge triplet consists of (head, relation, tail) along with their types and properties."
    "\n\nCONTEXT AWARENESS:"
    "\n- Focus on startup-related entities: founders, companies, investors, concepts"
    "\n- Identify key entrepreneurship principles and advice"
    "\n- Capture relationships between people, organizations, and ideas"
    "\n- Extract time-sensitive information when available (founded_date, funding_rounds, etc.)"
    "\n---------------------\n"
    "INITIAL ONTOLOGY:\n"
    "Entity Types: {allowed_entity_types}\n"
    "Entity Properties: {allowed_entity_properties}\n"
    "Relation Types: {allowed_relation_types}\n"
    "Relation Properties: {allowed_relation_properties}\n"
    "\n"
    "Use these types as a starting point, but introduce new types if necessary based on the context.\n"
    "If the Entity Properties, Relation Properties contain property 'description', generate a context aware detailed description, which will have some uniqe non generic information addition"
    "\n"
    "GUIDELINES:\n"
    "- Output in JSON format: [{{'head': '', 'head_type': '', 'head_props': {{...}}, 'relation': '', 'relation_props': {{...}}, 'tail': '', 'tail_type': '', 'tail_props': {{...}}}}]\n"
    "- Use the most complete form for entities (e.g., 'United States of America' instead of 'USA') but where its ambiguous, use the entity as it is\n"
    "- Keep entities concise\n"
    "- While writing description property for entities and relations keep context in mind and just dont write the description of the entity or relation, but the description of the entity or relation in the context of the text\n"
    "- Ensure the knowledge graph is coherent and easily understandable\n"
    "- While extracting relation, use singular form of the relation. Use EXPAND instead of EXPANDS or EXPECT instead of EXPECTS\n"
    "- The goal is to make relations as generics as possible, so that there are less duplicate relations in the graph, which have same meaning\n"
    "- If there are two names present in the text treat them as separate entities. For example Jessica Livingston and Robert Morris then they are two separate entities Jessica Livingston, Robert Morris\n"
    "- Focus on startup-specific metrics and relationships (funding rounds, valuations, mentor relationships)\n"
    "- Capture temporal aspects of relationships when mentioned (founding dates, acquisition dates)\n"
    "- Include relevant contextual properties (industry sector, technology stack, market focus)\n"
    "---------------------\n"
    "EXAMPLE:\n"
    "Text Input: \nTim Cook, CEO of Apple Inc., announced the new Apple Watch that monitors heart health. "
    "UC Berkeley researchers studied the benefits of apples.\n"
    "Example Output:\n"
    "[{{'head': 'Tim Cook', 'head_type': 'PERSON', 'head_props': {{'description': 'Technology executive who made the product announcement for Apple Watch, demonstrating leadership in health-focused technology initiatives'}}, 'relation': 'CEO_OF', 'relation_props': {{'description': 'Executive leadership role involving product announcements and strategic health technology initiatives'}}, 'tail': 'Apple Inc.', 'tail_type': 'COMPANY', 'tail_props': {{'description': 'Technology company expanding into health monitoring through wearable devices'}}}},\n"
    " {{'head': 'Apple Inc.', 'head_type': 'COMPANY', 'head_props': {{'description': 'Company developing health-focused consumer technology products under Tim Cook's leadership'}}, 'relation': 'PRODUCE', 'relation_props': {{'description': 'Strategic initiative to enter health monitoring market through consumer devices'}}, 'tail': 'Apple Watch', 'tail_type': 'PRODUCT', 'tail_props': {{'description': 'Health-focused smartwatch representing Apple's expansion into medical monitoring technology'}}}},\n"
    " {{'head': 'Apple Watch', 'head_type': 'PRODUCT', 'head_props': {{'description': 'Wearable device specifically designed to track and monitor user health metrics'}}, 'relation': 'MONITOR', 'relation_props': {{'description': 'Continuous health monitoring capability focusing on cardiac metrics'}}, 'tail': 'heart health', 'tail_type': 'HEALTH_METRIC', 'tail_props': {{'description': 'Critical health metric monitored through Apple Watch's advanced sensors'}}}},\n"
    " {{'head': 'UC Berkeley', 'head_type': 'UNIVERSITY', 'head_props': {{'description': 'Academic institution conducting research on nutritional benefits and health impacts'}}, 'relation': 'STUDY', 'relation_props': {{'description': 'Academic research focusing on health benefits and nutritional value analysis'}}, 'tail': 'benefits of apples', 'tail_type': 'RESEARCH_TOPIC', 'tail_props': {{'description': 'Scientific investigation into the health advantages and nutritional properties of apples'}}}}]\n"
    "---------------------\n"
    "MAKE SURE TO FOLLOW THE EXAMPLE FORMAT STRICTLY FOR EACH KNOWLEDGE TRIPLET AND GIVE THE OUTPUT IN JSON LIST FORMAT ONLY AS ITS PARSED USING CODE AND IT NEEDS TO MATCH THE EXAMPLE FORMAT STRICTLY"
    "Text: {text}\n"
    "Output:\n"
)

In [12]:
from typing import Any, Dict, List, Tuple
import re
import json
from llama_index.core.graph_stores.types import (
    EntityNode,
    Relation,
)

def parse_dynamic_triplets_with_props(
    llm_output: str,
) -> List[Tuple[EntityNode, Relation, EntityNode]]:
    """
    Parse the LLM output and convert it into a list of entity-relation-entity triplets.
    This function is flexible and can handle various output formats.

    Args:
        llm_output (str): The output from the LLM, which may be JSON-like or plain text.

    Returns:
        List[Tuple[EntityNode, Relation, EntityNode]]: A list of triplets.
    """
    triplets = []

    try:
        #first extract the json from markdown
        llm_output = re.search(r'```json\s*([\s\S]*)\s*```', llm_output).group(1)
        # Attempt to parse the output as JSON
        data = json.loads(llm_output)
        for item in data:
            head = item.get("head")
            head_type = item.get("head_type")
            head_props = item.get("head_props", {})
            relation = item.get("relation")
            relation_props = item.get("relation_props", {})
            tail = item.get("tail")
            tail_type = item.get("tail_type")
            tail_props = item.get("tail_props", {})

            if head and head_type and relation and tail and tail_type:
                head_node = EntityNode(
                    name=head, label=head_type, properties=head_props
                )
                tail_node = EntityNode(
                    name=tail, label=tail_type, properties=tail_props
                )
                relation_node = Relation(
                    source_id=head_node.id,
                    target_id=tail_node.id,
                    label=relation,
                    properties=relation_props,
                )
                triplets.append((head_node, relation_node, tail_node))
    except json.JSONDecodeError:
        # Flexible pattern to match the key-value pairs for head, head_type, head_props, relation, relation_props, tail, tail_type, and tail_props
        pattern = r'[\{"\']head[\}"\']\s*:\s*[\{"\'](.*?)[\}"\']\s*,\s*[\{"\']head_type[\}"\']\s*:\s*[\{"\'](.*?)[\}"\']\s*,\s*[\{"\']head_props[\}"\']\s*:\s*\{(.*?)\}\s*,\s*[\{"\']relation[\}"\']\s*:\s*[\{"\'](.*?)[\}"\']\s*,\s*[\{"\']relation_props[\}"\']\s*:\s*\{(.*?)\}\s*,\s*[\{"\']tail[\}"\']\s*:\s*[\{"\'](.*?)[\}"\']\s*,\s*[\{"\']tail_type[\}"\']\s*:\s*[\{"\'](.*?)[\}"\']\s*,\s*[\{"\']tail_props[\}"\']\s*:\s*\{(.*?)\}\s*'

        # Find all matches in the output
        matches = re.findall(pattern, llm_output)

        for match in matches:
            (
                head,
                head_type,
                head_props,
                relation,
                relation_props,
                tail,
                tail_type,
                tail_props,
            ) = match

            # Use more robust parsing for properties
            def parse_props(props_str: str) -> Dict[str, Any]:
                try:
                    # Handle mixed quotes and convert to a proper dictionary
                    props_str = props_str.replace("'", '"')
                    return json.loads(f"{{{props_str}}}")
                except json.JSONDecodeError:
                    return {}

            head_props_dict = parse_props(head_props)
            relation_props_dict = parse_props(relation_props)
            tail_props_dict = parse_props(tail_props)

            head_node = EntityNode(
                name=head, label=head_type, properties=head_props_dict
            )
            tail_node = EntityNode(
                name=tail, label=tail_type, properties=tail_props_dict
            )
            relation_node = Relation(
                source_id=head_node.id,
                target_id=tail_node.id,
                label=relation,
                properties=relation_props_dict,
            )
            triplets.append((head_node, relation_node, tail_node))
    return triplets

In [13]:
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor

dyn_llm_path_extractor = DynamicLLMPathExtractor(
    llm=Settings.llm,
    allowed_entity_props=["description"],
    allowed_relation_props=["description"],
    extract_prompt=prompt_template,
    parse_fn=parse_dynamic_triplets_with_props,
)

In [32]:
# from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
# neo4j_vector = Neo4jVectorStore(
#     username="neo4j",
#     password="12345678",
#     url="bolt://localhost:7687",
#     database="graphrag-openai-custom-extractor-prompt-gemini",
#     hybrid_search = True,
#     embedding_dimension=768
# )


In [19]:
from llama_index.core import PropertyGraphIndex
# from llama_index.core.indices.property_graph import SimpleLLMPathExtractor

# index = PropertyGraphIndex(
#     nodes_list[0],
#     embed_model=Settings.embed_model,
#     kg_extractors=[dyn_llm_path_extractor],
#     property_graph_store=graph_store,
#     show_progress=True,
# )

index = PropertyGraphIndex.from_existing(
    # vector_store=neo4j_vector,
    property_graph_store=graph_store,
    llm=Settings.llm,
    embed_model=Settings.embed_model,
    kg_extractors=[dyn_llm_path_extractor],
    show_progress=True,
)

In [16]:
index._insert_nodes(nodes_list[1])

In [16]:
index._insert_nodes(nodes_list[2])

Extracting and inferring knowledge graph from text:   0%|          | 0/10 [00:00<?, ?it/s]

🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01938d03-8d71-7e62-9fb8-a23c895e2d58
🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01938d03-cb6f-77d0-ade4-efd95db4448a
🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01938d04-023a-77f0-bfbc-811641f07c49
🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01938d04-3438-7122-b775-dc4a8fbf9d6b
🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01938d04-5d4a-7012-b074-8ba13f759d70
🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01938d04-9103-7d70-b133-1e1927d1d4b4
🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01938d04-c301-7c12-ae1d-07c18105ea35
🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01938d04-fa55-7f21-a6f9-3ea45d366fde
🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01938d05-301b-79a1-a77d-d40b3496d8c5


Extracting and inferring knowledge graph from text: 100%|██████████| 10/10 [02:09<00:00, 12.96s/it]


🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01938d05-61d2-7472-88d4-2534d1d15a5d


Generating embeddings: 100%|██████████| 5/5 [00:03<00:00,  1.58it/s]


[TextNode(id_='ff705b59-8f1e-4614-afbb-66303e529680', embedding=[-0.011374777, -0.079265736, -0.03172533, -0.037470292, 0.05025024, 0.040209018, 0.005124908, 0.005554039, 0.0055763223, 0.017367674, 0.023351206, -0.0012620416, -0.0025339427, -0.00095113233, 0.039345223, -0.028447852, -0.0050922874, 0.016701078, -0.008945524, -0.0300458, 0.0026624426, -0.0023660085, -0.0006174729, -0.039336823, -0.0023937728, -0.020500032, 0.05266727, -0.0560316, -0.0048279734, 0.06285275, -0.051348962, 0.07096855, -0.08576716, -0.010622374, 0.015585635, -0.035101146, 0.017711878, 0.0048641595, 0.011984316, 0.034206506, -0.043311063, -0.04775628, -0.044865765, -0.015053323, 0.018037211, 0.019132026, -0.0077078794, -0.022303194, 0.042800065, -0.062107082, -0.048599552, -0.016613517, 0.04635088, -0.0046377815, 0.005374716, -0.023693226, 0.04705154, -0.10416063, -0.06839797, 0.04474311, 0.010550733, -0.0077320463, 0.023699226, -0.014945423, 0.04090456, -0.04369151, -0.058957487, 0.026428496, 0.06511561, 0.0

In [17]:
# import time
# index._insert_nodes(nodes_list[3])
# time.sleep(60)
# index._insert_nodes(nodes_list[4])
# time.sleep(60)
# index._insert_nodes(nodes_list[5])
# time.sleep(60)
# index._insert_nodes(nodes_list[6])
# time.sleep(60)
# index._insert_nodes(nodes_list[7])

Extracting and inferring knowledge graph from text: 100%|██████████| 1/1 [00:11<00:00, 11.12s/it]


🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01938d0a-c92f-7792-a4c0-133840cde408


Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.41s/it]


[TextNode(id_='3fe54946-db9e-487c-8570-e01e79e27f08', embedding=[-0.013015766, -0.03615942, -0.0657989, -0.046315454, 0.03470951, 0.008519951, -0.006096066, 0.016339034, 0.036732428, 0.03029259, 0.02838194, 0.00940741, -0.05408982, -0.0046703992, 0.068405986, -0.039870407, 0.0013439719, 0.030576656, -0.031527698, -0.052709077, -0.029399386, 0.01797618, 0.030843675, -0.011176268, -0.01051241, -0.006781152, 0.046946857, -0.06765896, 0.010598244, 0.004466792, -0.0402693, 0.049643487, -0.07323995, 0.0390673, 0.002017495, -0.05124213, -0.01912209, 0.008916144, -0.022323443, -0.0105410945, -0.008113085, -0.029571712, -0.054811932, -0.0039814943, -0.022089211, -0.0040400447, -0.0034895204, 0.019759232, -0.007055731, -0.06114749, 0.0100573925, 0.03412498, 0.07903026, -0.030521745, 0.0048228507, -0.0093593225, 0.07555433, -0.03840786, -0.03942476, 0.027020171, 0.0033294244, -0.009529802, -0.015105269, -0.009140847, 0.01126444, -0.05417937, -0.074751236, 0.017331691, 0.049847312, 0.025802892, -0

In [18]:
# graph_store.structured_query("""
# CREATE VECTOR INDEX entity IF NOT EXISTS
# FOR (m:`__Entity__`)
# ON m.embedding
# OPTIONS {indexConfig: {
#  `vector.dimensions`: 1536,
#  `vector.similarity_function`: 'cosine'
# }}
# """)

[]

In [19]:
# similarity_threshold = 0.9
# word_edit_distance = 4
# data = graph_store.structured_query("""
# MATCH (e:__Entity__)
# CALL {
#   WITH e
#   CALL db.index.vector.queryNodes('entity', 10, e.embedding)
#   YIELD node, score
#   WITH node, score
#   WHERE score > toFLoat($cutoff)
#       AND (toLower(node.name) CONTAINS toLower(e.name) OR toLower(e.name) CONTAINS toLower(node.name)
#            OR apoc.text.distance(toLower(node.name), toLower(e.name)) < $distance)
#       AND labels(e) = labels(node)
#   WITH node, score
#   ORDER BY node.name
#   RETURN collect(node) AS nodes
# }
# WITH distinct nodes
# WHERE size(nodes) > 1
# WITH collect([n in nodes | n.name]) AS results
# UNWIND range(0, size(results)-1, 1) as index
# WITH results, index, results[index] as result
# WITH apoc.coll.sort(reduce(acc = result, index2 IN range(0, size(results)-1, 1) |
#         CASE WHEN index <> index2 AND
#             size(apoc.coll.intersection(acc, results[index2])) > 0
#             THEN apoc.coll.union(acc, results[index2])
#             ELSE acc
#         END
# )) as combinedResult
# WITH distinct(combinedResult) as combinedResult
# // extra filtering
# WITH collect(combinedResult) as allCombinedResults
# UNWIND range(0, size(allCombinedResults)-1, 1) as combinedResultIndex
# WITH allCombinedResults[combinedResultIndex] as combinedResult, combinedResultIndex, allCombinedResults
# WHERE NOT any(x IN range(0,size(allCombinedResults)-1,1) 
#     WHERE x <> combinedResultIndex
#     AND apoc.coll.containsAll(allCombinedResults[x], combinedResult)
# )
# RETURN combinedResult  
# """, param_map={'cutoff': similarity_threshold, 'distance': word_edit_distance})
# for row in data:
#     print(row)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (e) { ... }} {position: line: 3, column: 1, offset: 22} for query: "\nMATCH (e:__Entity__)\nCALL {\n  WITH e\n  CALL db.index.vector.queryNodes('entity', 10, e.embedding)\n  YIELD node, score\n  WITH node, score\n  WHERE score > toFLoat($cutoff)\n      AND (toLower(node.name) CONTAINS toLower(e.name) OR toLower(e.name) CONTAINS toLower(node.name)\n           OR apoc.text.distance(toLower(node.name), toLower(e.name)) < $distance)\n      AND labels(e) = labels(node)\n  WITH node, score\n  ORDER BY node.name\n  RETURN collect(node) AS nodes\n}\nWITH distinct nodes\nWHERE size(nodes) > 1\nWITH collect([n in nodes | n.name]) AS results\nUNWIND range(0, size(results)-1, 1) as index\

{'combinedResult': ['Founders', 'VCs, Founders, Outside Member', 'Viaweb Founders', 'founders']}
{'combinedResult': ['Apple Board of Directors', 'Board of Directors']}
{'combinedResult': ['Paul Graham', 'Paul Graham (implied)']}
{'combinedResult': ['Steve Wozniak', 'Wozniak']}
{'combinedResult': ['Undergrads', 'undergrads']}
{'combinedResult': ['Robert', 'Robert Morris']}
{'combinedResult': ['agreement', 'restrictive employee agreement']}
{'combinedResult': ['Early-Stage Startups', 'Startup']}
{'combinedResult': ['aspiring startup founders', 'startup founders']}
{'combinedResult': ['20 year old founders', '24 year old founders']}
{'combinedResult': ['Author', 'author']}
{'combinedResult': ['Adobe', 'Apple']}
{'combinedResult': ['Undergraduates', 'undergraduates']}
{'combinedResult': ['1959', '1998']}
{'combinedResult': ['David Filo', 'David Filo and Jerry Yang']}
{'combinedResult': ['Bill Gates', 'Bill Gates (age 19)']}
{'combinedResult': ['Startups (1990s)', 'Startups (2020s)']}
{'com

In [43]:
query_engine = index.as_query_engine(
    include_text=True,
    include_properties=True,
    similarity_top_k=10,
    limit=50,
    similarity_score=0.75,
    hybrid_search=True
)
response = query_engine.query(
    "How did the example of Google influence later startups and investor behavior, particularly regarding the role of founders in company leadership? Give a detailed answer, include all the relevant information available. It is okay if the answer is lengthy. Its is very critical that the answers are written using information available in the provided data. Provide file names in the answer as references at the end of the answer."
)
print(response)

🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/019391d3-38f6-7c62-bb38-f5a06c0aa51f
🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/019391d3-4d17-7ca2-ac16-c4b1e22ce6ca
Google's founders successfully resisted investor pressure to immediately appoint an experienced CEO, instead delaying for a year and ultimately selecting a CEO with a PhD in computer science.  This demonstrated increasing founder control and influenced later startups by showing that founders could push back against investor demands for experienced leadership.  The success of Google, despite its young and inexperienced founders, further supported this trend.  This example suggests a shift towards valuing founders' technical expertise and allowing them to maintain leadership roles, even against investor preferences for more seasoned executives.  The prediction is that future executives installed by venture capitalists will increasingly be COOs rather than CEOs, with founders directly managing engineering.  This

In [39]:
print(response.source_nodes[1])

Node ID: 1f39a1b8-a229-41c9-bfaa-f0bec95c4cad
Text: Here are some facts extracted from the provided text:  Google
({'creation_date': '2024-11-26', 'last_modified_date': '2024-11-20',
'file_size': 26756, 'file_path': 'D:\\projects\\graphrag-
poc\\data\\selection 1\\Hiring is Obsolete (HIVE).txt', 'description':
'One of the three major internet companies in May 2005, alongside
Yahoo and Microsoft, n...
Score:  0.892



In [25]:
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
neo4j_vector = Neo4jVectorStore(
    username="neo4j",
    password="12345678",
    url="bolt://localhost:7687",
    database="graphrag-openai-custom-extractor-prompt-gemini",
    hybrid_search = True,
    embedding_dimension=768
)


True


In [92]:
from llama_index.core.indices.property_graph import (
    LLMSynonymRetriever,
    VectorContextRetriever
)
from graphrag_poc.custom_llama_index.vector_retriever import CustomVectorContextRetriever

vector_retriever = VectorContextRetriever(
     graph_store,
     embed_model=Settings.embed_model,
    #  similarity_top_k=4,
     path_depth=3,
    #  include_text=True,
    #  include_properties=True,
     similarity_score=0.7
)

llm_synonym_retriever = LLMSynonymRetriever(
    graph_store=graph_store,
    include_text=False,
    # include_properties=True,
    max_keywords=20,
    llm=Settings.llm,
)

custom_vector_retriever = CustomVectorContextRetriever(
     graph_store,
     embed_model=Settings.embed_model,
    #  similarity_top_k=4,
     path_depth=3,
     include_text=False,
    #  include_properties=True,
     similarity_score=0.7
)


In [81]:
query_engine = index.as_query_engine(
    sub_retrievers=[vector_retriever, llm_synonym_retriever],
    # include_text=True,
    # include_properties=True,
    similarity_top_k=10,
    limit=100,
    similarity_score=0.75,
)
response = query_engine.query(
    "How did the example of Google influence later startups and investor behavior, particularly regarding the role of founders in company leadership? Give a detailed answer, include all the relevant information available. It is okay if the answer is lengthy. Its is very critical that the answers are written using information available in the provided data."
)
print(response)

🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/019391e6-4ce6-7e12-a566-7e7fb2665d81
🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/019391e6-5311-7c90-8f0a-e5dd5350c025
Google's founders successfully resisted investor pressure to immediately appoint an experienced CEO.  Instead, they delayed for a year and ultimately chose a CEO with a PhD in computer science, prioritizing technical expertise. This demonstrated that founders could retain significant control over leadership decisions, even when facing investor pressure.  This example suggests that future founders might be better able to push back against investors' demands for specific leadership appointments, particularly those who might not prioritize the founders' vision.  The author predicts that, in the future, investor-installed executives will increasingly be Chief Operating Officers (COOs) rather than CEOs, allowing founders to directly manage engineering and the COO to manage the rest of the company.  Google's succes

In [93]:
from llama_index.core.base.response.schema import RESPONSE_TYPE

query_engine = index.as_query_engine(
    sub_retrievers=[custom_vector_retriever, llm_synonym_retriever],
    # include_text=True,
    # include_properties=True,
    similarity_top_k=10,
    limit=100,
    similarity_score=0.75,
)
response: RESPONSE_TYPE = query_engine.query(
    "Tell me all about yahoo. It is okay if the answer is lengthy. Its is very critical that the answers are written using information available in the provided data."
)
print(response)

🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01939562-58fa-7db1-96cc-504378a228f1
🍩 https://wandb.ai/vedmanivaidya/graphrag-poc/r/call/01939562-5f74-71c0-bfcf-26ef8b2eddb4
Yahoo was one of three major internet companies in May 2005, alongside Google and Microsoft.  It's notable for its relatively young founders, David Filo and Jerry Yang, who established the company in March 1995 while pursuing graduate studies.  This demonstrates an alternative to traditional employment.  Yahoo's founding illustrates the potential for success despite initial uncertainty felt by its founders.  The company's hiring preferences, as indicated by Zod Nazem, favored candidates with failed startup experience, valuing entrepreneurial experience even with setbacks.  Yahoo competed with venture capitalists for acquiring early-stage startups.  David Filo, a key figure at Yahoo, was known for his cost-conscious management style, even with a multi-billion dollar market cap.  However, Yahoo's failure to pri

In [94]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='10681b3f-31c3-4d02-90f0-f03e92639e61', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1f39a1b8-a229-41c9-bfaa-f0bec95c4cad', node_type=None, metadata={}, hash=None)}, metadata_template='{key}: {value}', metadata_separator='\n', text='Yahoo (Description: One of the three major internet companies in May 2005, alongside Google and Microsoft, notable for its relatively young founders.) -> FOUNDED_BY (Description: Founding of the company by individuals with an average age of 24, as mentioned in the essay.) -> Founders (average age 24) (Description: Group of founders with an average age of 24, used as an example to support the argument for younger entrepreneurs.)', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.9041211605072021),
 NodeWithScore(node

In [97]:
from llama_index.core.schema import NodeWithScore

response_nodes: List[NodeWithScore] = response.source_nodes

for node in response_nodes:
    print(node.get_content())
    print("--------------------------------")


Yahoo (Description: One of the three major internet companies in May 2005, alongside Google and Microsoft, notable for its relatively young founders.) -> FOUNDED_BY (Description: Founding of the company by individuals with an average age of 24, as mentioned in the essay.) -> Founders (average age 24) (Description: Group of founders with an average age of 24, used as an example to support the argument for younger entrepreneurs.)
--------------------------------
Google (Description: One of the three major internet companies in May 2005, alongside Yahoo and Microsoft, notable for its relatively young founders.) -> FOUNDED_BY (Description: Founding of the company by individuals with an average age of 24, as mentioned in the essay.) -> Founders (average age 24) (Description: Group of founders with an average age of 24, used as an example to support the argument for younger entrepreneurs.)
--------------------------------
Larry Page (Description: Programmer who, along with Sergey Brin, aimed